# Sub-Workflow Request Interception

## Overview

This notebook demonstrates **request interception** - showing how parent workflows can intercept and handle requests from sub-workflows before they reach external services.

### Email Validation Flow:

```
Parent Workflow:
  SmartEmailOrchestrator (start)
      |
      | EmailValidationRequest x3 (concurrent)
      v
  [ Sub-Workflow: WorkflowExecutor(EmailValidator) ]
      |
      | DomainCheckRequest (RequestInfoMessage)
      v
  Interception Decision:
      |
      |-- Known domain (example.com, company.com)
      |   -> SmartEmailOrchestrator.handle_domain_request()
      |   -> Send RequestResponse(data=True) back to sub-workflow
      |
      |-- Unknown domain (unknown.org)
          -> Forward to RequestInfoExecutor
          -> External service handles
          -> Response routed back to sub-workflow
```

### Key Concepts:

1. **RequestInfoMessage**: Special message type for external requests
2. **Typed Handler Interception**: `@handler` methods catch specific message types
3. **RequestResponse**: Correlates responses with original requests
4. **Request Forwarding**: Unhandled requests go to external services
5. **Request Correlation**: `request_id` and `source_executor_id` match responses
6. **Concurrent Isolation**: Multiple sub-workflows don't interfere

### What You Learn:

- Create RequestInfoMessage subclasses
- Intercept sub-workflow requests with typed handlers
- Conditionally handle or forward requests
- Correlate responses using request_id
- Handle external requests via RequestInfoExecutor
- Build intelligent caching/optimization layers

## Prerequisites

- Agent Framework installed: `pip install agent-framework`
- No external services required (simulated via RequestInfoExecutor)

## Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass

import os
from dotenv import load_dotenv
from agent_framework import (
    Executor,
    RequestInfoExecutor,
    RequestInfoMessage,
    RequestResponse,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowExecutor,
    handler,
)
# Load environment variables from .env file
load_dotenv('../../.env')


## Define Message Types

### DomainCheckRequest (RequestInfoMessage)

**Why subclass RequestInfoMessage?**
- Enables interception by parent executors
- Automatic request_id and source_executor_id tracking
- Framework handles request/response correlation
- Can be forwarded to external services if needed

In [ ]:
@dataclass
class EmailValidationRequest:
    """Request to validate an email address."""
    email: str


@dataclass
class DomainCheckRequest(RequestInfoMessage):
    """Request to check if a domain is approved.
    
    Subclasses RequestInfoMessage to enable:
    - Interception by parent executors
    - Automatic request correlation
    - External service forwarding
    """
    domain: str = ""


@dataclass
class ValidationResult:
    """Result of email validation."""
    email: str
    is_valid: bool
    reason: str


print("✓ Message types defined")

## Create Sub-Workflow Executor

### EmailValidator

**Demonstrates:**
- Sub-workflow makes external requests (DomainCheckRequest)
- Uses `_pending_emails` dict for request correlation
- Handles RequestResponse with proper request_id matching
- Sub-workflow is unaware of parent interception

In [ ]:
class EmailValidator(Executor):
    """Validates email addresses - doesn't know it's in a sub-workflow."""

    def __init__(self) -> None:
        """Initialize the EmailValidator executor."""
        super().__init__(id="email_validator")
        # Use a dict to track multiple pending emails by request_id
        self._pending_emails: dict[str, str] = {}

    @handler
    async def validate_request(
        self,
        request: EmailValidationRequest,
        ctx: WorkflowContext[DomainCheckRequest | ValidationResult, ValidationResult],
    ) -> None:
        """Validate an email address."""
        print(f"🔍 Sub-workflow validating email: {request.email}")

        # Extract domain
        domain = request.email.split("@")[1] if "@" in request.email else ""

        if not domain:
            print(f"❌ Invalid email format: {request.email}")
            result = ValidationResult(
                email=request.email, 
                is_valid=False, 
                reason="Invalid email format"
            )
            await ctx.yield_output(result)
            return

        print(f"🌐 Sub-workflow requesting domain check for: {domain}")
        # Request domain check
        domain_check = DomainCheckRequest(domain=domain)
        # Store the pending email with the request_id for correlation
        self._pending_emails[domain_check.request_id] = request.email
        await ctx.send_message(domain_check, target_id="email_request_info")

    @handler
    async def handle_domain_response(
        self,
        response: RequestResponse[DomainCheckRequest, bool],
        ctx: WorkflowContext[ValidationResult, ValidationResult],
    ) -> None:
        """Handle domain check response from RequestInfo with correlation."""
        approved = bool(response.data)
        domain = (
            response.original_request.domain
            if (hasattr(response, "original_request") and response.original_request)
            else "unknown"
        )
        print(f"📬 Sub-workflow received domain response for '{domain}': {approved}")

        # Find the corresponding email using the request_id
        request_id = (
            response.original_request.request_id
            if (hasattr(response, "original_request") and response.original_request)
            else None
        )
        if request_id and request_id in self._pending_emails:
            email = self._pending_emails.pop(request_id)  # Remove from pending
            result = ValidationResult(
                email=email,
                is_valid=approved,
                reason="Domain approved" if approved else "Domain not approved",
            )
            print(f"✅ Sub-workflow completing validation for: {email}")
            await ctx.yield_output(result)


print("✓ EmailValidator executor created")

## Create Parent Workflow Executor

### SmartEmailOrchestrator

**Demonstrates:**
- Typed handler for DomainCheckRequest interception
- Conditional handling vs. forwarding
- RequestResponse creation with proper correlation
- Result collection from sub-workflows

In [ ]:
class SmartEmailOrchestrator(Executor):
    """Parent orchestrator that can intercept domain checks."""

    approved_domains: set[str] = set()

    def __init__(self, approved_domains: set[str] | None = None):
        """Initialize the SmartEmailOrchestrator with approved domains.

        Args:
            approved_domains: Set of pre-approved domains
        """
        super().__init__(id="email_orchestrator", approved_domains=approved_domains)
        self._results: list[ValidationResult] = []

    @handler
    async def start_validation(
        self, 
        emails: list[str], 
        ctx: WorkflowContext[EmailValidationRequest]
    ) -> None:
        """Start validating a batch of emails."""
        print(f"📧 Starting validation of {len(emails)} email addresses")
        print("=" * 60)
        for email in emails:
            print(f"📤 Sending '{email}' to sub-workflow for validation")
            request = EmailValidationRequest(email=email)
            await ctx.send_message(request, target_id="email_validator_workflow")

    @handler
    async def handle_domain_request(
        self,
        request: DomainCheckRequest,
        ctx: WorkflowContext[RequestResponse[DomainCheckRequest, bool] | DomainCheckRequest],
    ) -> None:
        """Intercept domain check requests from sub-workflows."""
        print(f"🔍 Parent intercepting domain check for: {request.domain}")

        if request.domain in self.approved_domains:
            # Handle locally - create response
            print(f"✅ Domain '{request.domain}' is pre-approved locally!")
            
            # Send response back to sub-workflow
            response = RequestResponse(
                data=True, 
                original_request=request, 
                request_id=request.request_id
            )
            await ctx.send_message(response, target_id=request.source_executor_id)
        else:
            # Forward to external handler
            print(f"❓ Domain '{request.domain}' unknown, forwarding to external service...")
            await ctx.send_message(request)

    @handler
    async def collect_result(
        self, 
        result: ValidationResult, 
        ctx: WorkflowContext
    ) -> None:
        """Collect validation results from sub-workflow yielded output."""
        status_icon = "✅" if result.is_valid else "❌"
        print(f"📥 {status_icon} Validation result: {result.email} -> {result.reason}")
        self._results.append(result)

    @property
    def results(self) -> list[ValidationResult]:
        """Get the collected validation results."""
        return self._results


print("✓ SmartEmailOrchestrator executor created")

## Build Workflows

### Sub-Workflow Graph:

```
EmailValidator (start)
    ↓
RequestInfoExecutor ("email_request_info")
    ↓
EmailValidator (handle_domain_response)
```

### Parent Workflow Graph:

```
SmartEmailOrchestrator (start)
    ↓
WorkflowExecutor(validation_workflow)
    ↓
SmartEmailOrchestrator (collect_result + handle_domain_request)
    ↓ (if domain unknown)
RequestInfoExecutor ("main_request_info") → External
    ↓
WorkflowExecutor (route response back)
```

In [ ]:
print("🚀 Setting up sub-workflow with request interception...\n")

# Build the sub-workflow
email_validator = EmailValidator()
# Match the target_id used in EmailValidator ("email_request_info")
request_info = RequestInfoExecutor(id="email_request_info")

validation_workflow = (
    WorkflowBuilder()
    .set_start_executor(email_validator)
    .add_edge(email_validator, request_info)
    .add_edge(request_info, email_validator)
    .build()
)

print("✓ Sub-workflow created")

# Build the parent workflow with interception
orchestrator = SmartEmailOrchestrator(approved_domains={"example.com", "company.com"})
workflow_executor = WorkflowExecutor(validation_workflow, id="email_validator_workflow")
# Add a RequestInfoExecutor to handle forwarded external requests
main_request_info = RequestInfoExecutor(id="main_request_info")

main_workflow = (
    WorkflowBuilder()
    .set_start_executor(orchestrator)
    .add_edge(orchestrator, workflow_executor)
    .add_edge(workflow_executor, orchestrator)  # For ValidationResult and DomainCheckRequest
    # Add edges for external request handling
    .add_edge(orchestrator, main_request_info)
    .add_edge(main_request_info, workflow_executor)  # Route external responses to sub-workflow
    .build()
)

print("✓ Parent workflow created with interception")

## Prepare Test Data

Testing with 3 scenarios:
1. **user@example.com** - Known domain, intercepted locally
2. **admin@company.com** - Known domain, intercepted locally
3. **guest@unknown.org** - Unknown domain, forwarded externally

In [ ]:
test_emails = [
    "user@example.com",  # Should be intercepted and approved
    "admin@company.com",  # Should be intercepted and approved
    "guest@unknown.org",  # Should be forwarded externally
]

print(f"✓ Prepared {len(test_emails)} test emails")

## Run Workflow

Watch the interception pattern:
- example.com → Intercepted by parent
- company.com → Intercepted by parent
- unknown.org → Forwarded to external service

In [ ]:
result = await main_workflow.run(test_emails)

## Handle External Requests

Any requests not intercepted by the parent are collected as RequestInfoEvents.
We simulate external service responses and send them back.

In [ ]:
request_events = result.get_request_info_events()

if request_events:
    print(f"\n🌐 Handling {len(request_events)} external request(s)...")
    for event in request_events:
        if event.data and hasattr(event.data, "domain"):
            print(f"🔍 External domain check needed for: {event.data.domain}")

    # Simulate external responses
    external_responses: dict[str, bool] = {}
    for event in request_events:
        # Simulate external domain checking
        if event.data and hasattr(event.data, "domain"):
            domain = event.data.domain
            # Let's say unknown.org is actually approved externally
            approved = domain == "unknown.org"
            print(f"🌐 External service response for '{domain}': {'APPROVED' if approved else 'REJECTED'}")
            external_responses[event.request_id] = approved

    # Send external responses
    await main_workflow.send_responses(external_responses)
else:
    print("\n🎯 All requests were intercepted and handled locally!")

## Display Results Summary

In [ ]:
print("\n📊 Final Results Summary:")
print("=" * 60)

for result in orchestrator.results:
    status = "✅ VALID" if result.is_valid else "❌ INVALID"
    print(f"{status} {result.email}: {result.reason}")

print(f"\n🏁 Processed {len(orchestrator.results)} emails total")

## Expected Output Pattern

```
🚀 Setting up sub-workflow with request interception...

✓ Sub-workflow created
✓ Parent workflow created with interception

📧 Starting validation of 3 email addresses
============================================================
📤 Sending 'user@example.com' to sub-workflow for validation
📤 Sending 'admin@company.com' to sub-workflow for validation
📤 Sending 'guest@unknown.org' to sub-workflow for validation
🔍 Sub-workflow validating email: user@example.com
🌐 Sub-workflow requesting domain check for: example.com
🔍 Parent intercepting domain check for: example.com
✅ Domain 'example.com' is pre-approved locally!
📬 Sub-workflow received domain response for 'example.com': True
✅ Sub-workflow completing validation for: user@example.com
📥 ✅ Validation result: user@example.com -> Domain approved
🔍 Sub-workflow validating email: admin@company.com
🌐 Sub-workflow requesting domain check for: company.com
🔍 Parent intercepting domain check for: company.com
✅ Domain 'company.com' is pre-approved locally!
📬 Sub-workflow received domain response for 'company.com': True
✅ Sub-workflow completing validation for: admin@company.com
📥 ✅ Validation result: admin@company.com -> Domain approved
🔍 Sub-workflow validating email: guest@unknown.org
🌐 Sub-workflow requesting domain check for: unknown.org
🔍 Parent intercepting domain check for: unknown.org
❓ Domain 'unknown.org' unknown, forwarding to external service...

🌐 Handling 1 external request(s)...
🔍 External domain check needed for: unknown.org
🌐 External service response for 'unknown.org': APPROVED
📬 Sub-workflow received domain response for 'unknown.org': True
✅ Sub-workflow completing validation for: guest@unknown.org
📥 ✅ Validation result: guest@unknown.org -> Domain approved

📊 Final Results Summary:
============================================================
✅ VALID user@example.com: Domain approved
✅ VALID admin@company.com: Domain approved
✅ VALID guest@unknown.org: Domain approved

🏁 Processed 3 emails total
```

## Key Takeaways

### 1. RequestInfoMessage Pattern

```python
@dataclass
class DomainCheckRequest(RequestInfoMessage):
    """Subclass RequestInfoMessage for interception."""
    domain: str = ""
```

**Automatic Features:**
- `request_id`: Unique identifier for correlation
- `source_executor_id`: Tracks which executor sent it
- Framework routing: Can be intercepted or forwarded
- Type-based handling: Executors can filter by message type

### 2. Typed Handler Interception

```python
class Orchestrator(Executor):
    @handler
    async def handle_domain_request(
        self,
        request: DomainCheckRequest,  # Type-specific!
        ctx: WorkflowContext[...],
    ) -> None:
        # This handler ONLY receives DomainCheckRequest messages
        ...
```

**Routing Logic:**
- Framework inspects message type
- Calls handlers with matching type signature
- Multiple handlers can process same message
- Enables specialized interceptors

### 3. Conditional Handling vs. Forwarding

```python
@handler
async def handle_request(self, request: MyRequest, ctx) -> None:
    if can_handle_locally(request):
        # Handle locally - send response
        response = RequestResponse(
            data=my_result,
            original_request=request,
            request_id=request.request_id
        )
        await ctx.send_message(response, target_id=request.source_executor_id)
    else:
        # Forward to external service
        await ctx.send_message(request)
```

**Decision Criteria:**
- Cache hits
- Known patterns
- Policy rules
- Resource availability

### 4. RequestResponse Correlation

```python
# Create response
response = RequestResponse(
    data=True,                      # Response data
    original_request=request,        # Links to original request
    request_id=request.request_id    # Correlation ID
)

# Send to original requester
await ctx.send_message(response, target_id=request.source_executor_id)
```

**Correlation Fields:**
- `request_id`: Matches response to request
- `source_executor_id`: Where request came from
- `original_request`: Full request object
- `data`: Response payload

### 5. Request Tracking in Sub-Workflows

```python
class SubWorkflowExecutor(Executor):
    def __init__(self):
        super().__init__(id="sub_executor")
        # Track pending requests by request_id
        self._pending: dict[str, MyData] = {}
    
    @handler
    async def start(self, data: MyData, ctx) -> None:
        request = MyRequest(param=data.value)
        # Store for later correlation
        self._pending[request.request_id] = data
        await ctx.send_message(request)
    
    @handler
    async def handle_response(self, response: RequestResponse[MyRequest, Result], ctx) -> None:
        # Retrieve original data using request_id
        request_id = response.original_request.request_id
        if request_id in self._pending:
            original_data = self._pending.pop(request_id)
            # Process with original context
            ...
```

**Why Track?**
- Concurrent requests need correlation
- Original context required for processing
- Prevents response confusion
- Enables out-of-order handling

### 6. External Request Handling

```python
# In workflow graph
main_workflow = (
    WorkflowBuilder()
    .add_edge(orchestrator, main_request_info)  # Forwarded requests
    .add_edge(main_request_info, workflow_executor)  # Responses back
    .build()
)

# After workflow run
result = await main_workflow.run(data)
request_events = result.get_request_info_events()

if request_events:
    # Handle external requests
    responses = {}
    for event in request_events:
        responses[event.request_id] = external_service(event.data)
    
    # Send responses back
    await main_workflow.send_responses(responses)
```

### 7. Multi-Level Interception

```python
# Multiple executors can intercept same request type
class CacheLayer(Executor):
    @handler
    async def check_cache(self, request: DataRequest, ctx) -> None:
        if request.key in self.cache:
            # Handle from cache
            ...
        else:
            # Forward to next layer
            await ctx.send_message(request)

class DatabaseLayer(Executor):
    @handler
    async def check_db(self, request: DataRequest, ctx) -> None:
        if data := self.db.get(request.key):
            # Handle from database
            ...
        else:
            # Forward to external API
            await ctx.send_message(request)
```

**Layered Pattern:**
- Cache → Database → External API
- Each layer can handle or forward
- Optimizes response time
- Reduces external calls

### 8. Production Patterns

#### Caching with Expiration
```python
class SmartCache(Executor):
    def __init__(self):
        super().__init__(id="cache")
        self._cache: dict[str, tuple[Any, float]] = {}
        self._ttl = 300  # 5 minutes
    
    @handler
    async def handle_request(self, request: Request, ctx) -> None:
        if request.key in self._cache:
            data, timestamp = self._cache[request.key]
            if time.time() - timestamp < self._ttl:
                # Cache hit
                response = RequestResponse(
                    data=data,
                    original_request=request,
                    request_id=request.request_id
                )
                await ctx.send_message(response, target_id=request.source_executor_id)
                return
        
        # Cache miss or expired
        await ctx.send_message(request)
```

#### Rate Limiting
```python
class RateLimiter(Executor):
    @handler
    async def handle_request(self, request: APIRequest, ctx) -> None:
        if self._check_rate_limit(request.api_key):
            # Within limits - forward
            await ctx.send_message(request)
        else:
            # Rate limited - reject
            response = RequestResponse(
                data=None,
                error="Rate limit exceeded",
                original_request=request,
                request_id=request.request_id
            )
            await ctx.send_message(response, target_id=request.source_executor_id)
```

#### Circuit Breaker
```python
class CircuitBreaker(Executor):
    def __init__(self):
        super().__init__(id="circuit_breaker")
        self._failure_count = 0
        self._threshold = 5
        self._is_open = False
    
    @handler
    async def handle_request(self, request: ExternalRequest, ctx) -> None:
        if self._is_open:
            # Circuit open - use fallback
            response = RequestResponse(
                data=self._get_fallback_data(request),
                original_request=request,
                request_id=request.request_id
            )
            await ctx.send_message(response, target_id=request.source_executor_id)
        else:
            # Circuit closed - try external
            await ctx.send_message(request)
```